### Importing packages

In [86]:
import ansys.fluent.core as pyfluent
meshing_session = pyfluent.launch_fluent(precision="double", processor_count=4, mode="meshing", show_gui=False)
# set start_tracript to False for now? product_version="23.2.0" is also an argument that can be passed
workflow = meshing_session.workflow
meshing = meshing_session.meshing

### Initializing workflow for meshing

In [87]:
workflow.InitializeWorkflow(WorkflowType=r'Watertight Geometry')
meshing.GlobalSettings.LengthUnit.set_state(r'um')
meshing.GlobalSettings.AreaUnit.set_state(r'um^2')
meshing.GlobalSettings.VolumeUnit.set_state(r'um^3')

### Import geometry

Replace file_location with the actual address of the Static Mixer geometry file.
###### Eg: workflow.TaskObject['Import Geometry'].Arguments.set_state({r'FileName': r'D:/PyFluent/Static Mixer geometry.dsco',})


In [88]:
workflow.TaskObject['Import Geometry'].Arguments.set_state({r'FileName': r'E:\Summer Placement\Summer Placement Testbed\automationtest.scdoc',
                                                             "LengthUnit": "um",})
workflow.TaskObject['Import Geometry'].Execute() 

True

### Add local sizing

In [89]:
workflow.TaskObject['Add Local Sizing'].AddChildAndUpdate()

False

### Generate the Surface Mesh

In [90]:
workflow.TaskObject['Generate the Surface Mesh'].Execute()

True

### Describe geometry

In [91]:
workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=False)
workflow.TaskObject['Describe Geometry'].Arguments.set_state({r'SetupType': r'The geometry consists of only fluid regions with no voids',})
workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=True)
workflow.TaskObject['Describe Geometry'].Execute()

True

### Updating boundaries

In [92]:
workflow.TaskObject['Update Boundaries'].Execute()

True

### Updating regions

In [93]:
workflow.TaskObject['Update Regions'].Execute()

True

### Add boundary layers

In [94]:
workflow.TaskObject['Add Boundary Layers'].Arguments.set_state({r'LocalPrismPreferences': {r'Continuous': r'Stair Step',},})
workflow.TaskObject['Add Boundary Layers'].AddChildAndUpdate()

True

### Generate the volume Mesh

In [95]:
workflow.TaskObject["Generate the Volume Mesh"].Arguments = {"VolumeFill": "poly-hexcore",}

workflow.TaskObject['Generate the Volume Mesh'].Execute()

True

### Switch to solution

In [96]:
solver = meshing_session.switch_to_solver()

### Enabling energy

In [97]:
#solver.setup.models.energy = {"enabled" : False}

### Copying Water_liquid to materials panel

In [98]:
#solver.execute_tui(r'''/define/materials/copy fluid water-liquid ''')

### Assigning water_liquid to fluid domain

In [99]:
#solver.setup.cell_zone_conditions.fluid['fluid'] = {"material" : "water-liquid"}

### Assigning boudary conditons

In [100]:
solver.setup.boundary_conditions.velocity_inlet['inlet'].vmag = 3.78
# use this to find out available settings:
# solver.setup.boundary_conditions.wall['moving-wall']()
solver.setup.boundary_conditions.wall['moving-wall'].motion_bc = 'Moving Wall'
solver.setup.boundary_conditions.wall['moving-wall'].vmag = 3.78
solver.setup.boundary_conditions.wall['moving-wall'].wall_translation = [0, 0, 1]
#solver.setup.boundary_conditions.wall['moving-wall']()

### Residuals

In [101]:
solver.tui.solve.monitors.residual.plot("yes")

### Create surface report definition

In [102]:
solver.tui.solve.report_definitions.add(
    "outlet",
    "surface-massavg",
    "field",
    "velocity-magnitude",
    "surface-names",
    "outlet",
    "()",
    "quit",
)

### Hybrid Initialization

In [103]:
solver.solution.initialization.hybrid_initialize()

### Setting 20 iterations

In [104]:
solver.solution.run_calculation.iterate(iter_count = 30)

In [76]:
solver.tui.surface.plane(
    "yz-contour-plane",
    "yz-plane",
    "0"
)

RuntimeError: wta(1st) to string->symbol
Error Object: #f

In [ ]:
solver.tui.surface.rake_surface(
    "probe_rake",
    "0",        # x0
    "0",        # y0
    "0.0119",   # z0
    "0",        # x1
    "0.0015",   # y1
    "0.0119",   # z1
    "200",      # number of points
)

In [82]:
solver.tui.file.export.ascii(
    "TUI_test2.csv",        # file name
    "probe-rake",           # surface name (can be multiple)
    "()",                   # leave surface zone menu
    "yes",                    # delimiter
    "velocity-magnitude",   # ASCII scalar (can be multiple)
    "()",                   # leave scalar menu
    "no",                   # node values (no), cell center values (yes)
)

## Create velocity vectors
Create and display velocity vectors on the symmetry-xyplane plane.

- Set ``"vector"`` to ``"vector-vel"``.
- Set ``"style"`` to ``"arrow"``.
- Set ``"surface-list"`` to ``"symmetry-xyplane"``.
- Set ``"scale"`` to ``"4"``.
- Set ``"skip"`` to ``"2"``.



In [ ]:
solver.tui.display.objects.create(
    "vector",
    "vector-vel",
    "style",
    "arrow",
    "surface-list",
    "yz-contour-plane",
    "()",
    "scale",
    "scale-f",
    "4",
    "quit",
    "skip",
    "2",
    "quit",
)

## Display and save XY plot
Display and save an XY plot of the temperature profile across the centerline
of the outlet for the initial solution.



In [ ]:
solver.tui.display.objects.create(
    "xy",
    "xy-outlet-temp",
    "y-axis-function",
    "temperature",
    "surfaces-list",
    "z=0_outlet",
    "()",
    "quit",
)

### Exiting solver

In [105]:
solver.exit()